### 저장된 Yolo 불러오기

In [1]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'custom', path = './yolov5-master/exp3/weights/best.pt')
def net_yolo(img):
    h, w, c = img.shape
    pred_list = model(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    pred_list = pred_list.pandas().xyxy[0]
    result_list = []
    for cnt in range(len(pred_list)):
        x1 = int(pred_list.loc[cnt]['xmin'])
        y1 = int(pred_list.loc[cnt]['ymin'])
        x2 = int(pred_list.loc[cnt]['xmax'])
        y2 = int(pred_list.loc[cnt]['ymax'])
        x1_nor = max(1, int(pred_list.loc[cnt]['xmin'])) / w
        y1_nor = max(1, int(pred_list.loc[cnt]['ymin'])) / h
        x2_nor = max(1, int(pred_list.loc[cnt]['xmax'])) / w
        y2_nor = max(1, int(pred_list.loc[cnt]['ymax'])) / h
        conf = round(float(pred_list.loc[cnt]['confidence']), 3)
        pred = {'bbox' : [x1, y1, x2, y2], 'conf' : conf, 'bbox_nor' : [x1_nor, y1_nor, x2_nor, y2_nor]}
        result_list.append(pred)
    if len(result_list) > 0:
        result_list.sort(key = lambda x:x['conf'], reverse = True)
        result = result_list[0]                    
        crop_img = img[result['bbox'][1]:result['bbox'][3], result['bbox'][0]:result['bbox'][2]]
        return result['bbox_nor'], crop_img
    else:
        return [], []

Using cache found in C:\Users\nyan/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-9-15 Python-3.8.15 torch-1.10.1 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients
Adding AutoShape... 


### 저장된 Resnet 불러오기

In [2]:
def seed(seed = 1234):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
if torch.cuda.is_available() == True:
    device = 'cuda:0'
    print('현재 가상환경 GPU 사용 가능상태')
else:
    device = 'cpu'
    print('GPU 사용 불가능 상태')

현재 가상환경 GPU 사용 가능상태


In [3]:
from torchvision import models
import torch.nn as nn
from torchsummary import summary as Summary

seed()
model_resnet = models.resnet152(pretrained = True)
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_ftrs, 4)
model_resnet.load_state_dict(torch.load('./best_model_cnn.pth'))
net_cnn = model_resnet.to(device)
net_cnn.eval()
Summary(net_cnn.to(device), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

### 저장된 LSTM 불러오기

In [5]:
import torch.nn as nn

class skeleton_LSTM(nn.Module) :
    def __init__(self):
        super(skeleton_LSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size=8, hidden_size=128, num_layers=1, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=128, hidden_size=256, num_layers=1, batch_first=True)
        self.lstm3 = nn.LSTM(input_size=256, hidden_size=512, num_layers=1, batch_first=True)
        self.dropout1 = nn.Dropout(0.3)
        self.lstm4 = nn.LSTM(input_size=512, hidden_size=256, num_layers=1, batch_first=True)
        self.lstm5 = nn.LSTM(input_size=256, hidden_size=128, num_layers=1, batch_first=True)
        self.lstm6 = nn.LSTM(input_size=128, hidden_size=64, num_layers=1, batch_first=True)
        self.dropout2 = nn.Dropout(0.3)
        self.lstm7 = nn.LSTM(input_size=64, hidden_size=32, num_layers=1, batch_first=True)
        self.fc = nn.Linear(32,5)
    def forward(self, x) :
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = self.dropout1(x)
        x, _ = self.lstm4(x)
        x, _ = self.lstm5(x)
        x, _ = self.lstm6(x)
        x = self.dropout2(x)
        x, _ = self.lstm7(x)
        x = self.fc(x[:,-1,:])
        return x
    
net_lstm = skeleton_LSTM()
net_lstm.load_state_dict(torch.load('./model_lstm_logit_and_bbox.pth'))
net_lstm = net_lstm.to(device)
net_lstm.eval()

skeleton_LSTM(
  (lstm1): LSTM(8, 128, batch_first=True)
  (lstm2): LSTM(128, 256, batch_first=True)
  (lstm3): LSTM(256, 512, batch_first=True)
  (dropout1): Dropout(p=0.3, inplace=False)
  (lstm4): LSTM(512, 256, batch_first=True)
  (lstm5): LSTM(256, 128, batch_first=True)
  (lstm6): LSTM(128, 64, batch_first=True)
  (dropout2): Dropout(p=0.3, inplace=False)
  (lstm7): LSTM(64, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=5, bias=True)
)

### 레이블링 된 Test 데이터 돌려보기

In [7]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, Resize, Normalize
import numpy as np
import numpy as np
import torch.nn.functional as F

seed()
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
data_transforms = transforms.Compose([ToTensor(), Resize((224,224)), Normalize(mean, std)])
class dataset_cnn(Dataset):
    def __init__(self, img, label):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = data_transforms(img)
        self.img = [img]
        self.label = [label]
    def __getitem__(self, index):
        data = self.img[index]
        label = self.label[index]
        return data, label
    def __len__(self):
        return len(self.label)
    
class dataset_lstm(Dataset):
    def __init__(self, seq_list):
        self.X = []
        self.y = []
        for dic in seq_list :
            self.y.append(dic['key'])
            self.X.append(dic['value'])
    def __getitem__(self, index):
        data = self.X[index]
        label = self.y[index]
        return torch.Tensor(np.array(data)), torch.tensor(np.array(int(label)))
    def __len__(self):
        return len(self.X)

In [15]:
import os
import cv2
import natsort

def list_augmentation(input_list, goal_len):
    need_aug = goal_len - len(input_list)
    aug_tempo = need_aug / len(input_list)
    full_list, cnt = [], 0
    for _input in input_list:
        cnt += aug_tempo
        full_list.append(_input)
        while True:
            if cnt < 1: break
            full_list.append(_input)
            cnt -= 1
    while True:
        if len(full_list) <= goal_len: break
        del full_list[-1]
    return full_list
        
video_path = './test_label'
id_list, label_list = ['id'], ['label']
tp, fp, status = 0, 0, 'None'
folder_list = os.listdir(video_path)
folder_list = natsort.natsorted(folder_list)
for folder in folder_list:
    video_list = os.listdir('{}/{}'.format(video_path, folder))
    video_list = natsort.natsorted(video_list)
    for video_name in video_list:
        logit_and_bbox_list = []
        cap = cv2.VideoCapture('{}/{}/{}'.format(video_path, folder, video_name))
        if cap.isOpened():
            while True:
                ret, img = cap.read()
                if ret == True:
                    bbox, crop_img = net_yolo(img)
                    if len(crop_img) == 0: continue
                    dataset = dataset_cnn(crop_img, int(folder))
                    dataset = DataLoader(dataset)
                    for data, label in dataset:
                        data = data.to(device)
                        with torch.no_grad():
                            cnn_result = net_cnn(data)
                        cnn_logits = []
                        for logit in cnn_result[0]:
                            cnn_logits.append(logit.item())
                        logit_and_bbox = cnn_logits + bbox
                        logit_and_bbox_list.append(logit_and_bbox)
                else: break
        logit_and_bbox_list = list_augmentation(logit_and_bbox_list, 32)
        dataset_dict_list = [{'key':int(folder), 'value':logit_and_bbox_list}]
        dataset = dataset_lstm(dataset_dict_list)
        dataset = DataLoader(dataset)
        for data, label in dataset:
            data = data.to(device)
            with torch.no_grad():
                result = net_lstm(data)
            _, out = torch.max(result, 1)
            pred_class = out.item()
            id_list.append(video_name.split('.')[0])
            label_list.append(pred_class)
            if int(pred_class) == int(folder):
                tp += 1
                status = 'TP'
            else:
                fp += 1
                status = 'FP <-- 틀림'
            print('{} | GT: {} | Pred: {} | {}'.format(video_name, folder, pred_class, status))
print('Acc: {}'.format(tp / (tp+fp)))

TEST_002.mp4 | GT: 0 | Pred: 0 | TP
TEST_012.mp4 | GT: 0 | Pred: 0 | TP
TEST_032.mp4 | GT: 0 | Pred: 0 | TP
TEST_039.mp4 | GT: 0 | Pred: 0 | TP
TEST_047.mp4 | GT: 0 | Pred: 0 | TP
TEST_053.mp4 | GT: 0 | Pred: 0 | TP
TEST_058.mp4 | GT: 0 | Pred: 0 | TP
TEST_060.mp4 | GT: 0 | Pred: 0 | TP
TEST_067.mp4 | GT: 0 | Pred: 0 | TP
TEST_075.mp4 | GT: 0 | Pred: 0 | TP
TEST_079.mp4 | GT: 0 | Pred: 0 | TP
TEST_088.mp4 | GT: 0 | Pred: 0 | TP
TEST_093.mp4 | GT: 0 | Pred: 0 | TP
TEST_097.mp4 | GT: 0 | Pred: 0 | TP
TEST_104.mp4 | GT: 0 | Pred: 0 | TP
TEST_106.mp4 | GT: 0 | Pred: 0 | TP
TEST_107.mp4 | GT: 0 | Pred: 0 | TP
TEST_110.mp4 | GT: 0 | Pred: 0 | TP
TEST_126.mp4 | GT: 0 | Pred: 0 | TP
TEST_130.mp4 | GT: 0 | Pred: 0 | TP
TEST_131.mp4 | GT: 0 | Pred: 3 | FP <-- 틀림
TEST_136.mp4 | GT: 0 | Pred: 0 | TP
TEST_139.mp4 | GT: 0 | Pred: 0 | TP
TEST_140.mp4 | GT: 0 | Pred: 0 | TP
TEST_141.mp4 | GT: 0 | Pred: 0 | TP
TEST_150.mp4 | GT: 0 | Pred: 0 | TP
TEST_000.mp4 | GT: 1 | Pred: 1 | TP
TEST_008.mp4 | GT: 1 